# Implementação de POVMs no IBMQ

Estas notas são baseadas em (Y. S. Yordanov and C. H. W. Barnes, Implementation of a general single-qubit positive operator-valued measure on a circuit-based quantum computer, __Phys. Rev. A 100, 062317 (2019)__).

Os autores obtiveram uma forma de implementar POVM com $n$ elementos a partir da composição de operações básicas que implementam POVMs com 2 elementos.

Vale notar que na verdade os autores implementam medidas gerais. Ou seja, eles estão interessados em implementar um POVM com elementos $\{E_{j}\}=\{M_{j}^{\dagger}M_{j}\}$ em um sistema preparado inicialmente no estado $|\psi_{0}\rangle$ tal que
\begin{equation}
|\psi_{0}\rangle \xrightarrow[]{\text{obtém }M_{j}} \frac{M_{j}|\psi_{0}\rangle}{Pr(M_{j}|\psi_{0})}
\end{equation}
com
\begin{equation}
Pr(M_{j}|\psi_{0}) = ||M_{j}|\psi_{0}\rangle|| = \langle\psi_{0}|M_{j}^{\dagger}M_{j}|\psi_{0}\rangle.
\end{equation}

Usando um sistema auxiliar $B$, essa POVM pode ser implementada unitariamente:
\begin{equation}
U_{AB}|\psi_{0}\rangle_{A}\otimes|0\rangle_{B} = \sum_{j}(M_{j}|\psi_{0}\rangle_{A})\otimes|j\rangle_{B}.
\end{equation}

## POVMs de 1 qubit com 2 elementos
Começamos escrevendo o estado inicial na base computacional
\begin{equation}
|\psi_{0}\rangle = c_{0}|0\rangle_{A}+c_{1}|1\rangle_{A}.
\end{equation}
A ideia é manter em mente que queremos transformar o estado inicial 
\begin{equation}
|\Psi_{0}\rangle = |\psi_{0}\rangle_{A}\otimes|0\rangle_{B}
\end{equation}
em
\begin{equation}
|\Psi_{f}\rangle = (M_{1}|\psi_{0}\rangle_{A})\otimes|o_{1}\rangle_{B} + (M_{2}|\psi_{0}\rangle_{A})\otimes|o_{2}\rangle_{B},
\end{equation}
com $\{|o_{j}\rangle\}_{j=1}^{2}$ estados ortogonais do sistema auxiliar.

A primeira operação do protocolo é uma transformação unitária no sistema $A$:
\begin{align}
|\Psi_{1}\rangle & = (U_{A}\otimes\mathbb{I}_{B})|\Psi_{0}\rangle = (U_{A}|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} \\
& := (c_{0}'|0\rangle_{A}+c_{1}'|1\rangle_{A})\otimes|0\rangle_{B}.
\end{align}

<img src = 'qcircuit_povm2.png' width = '500'>

Seguindo aplicamos uma $R_{y}$ controlada, com o estado $|0\rangle$ como ativador
\begin{equation}
C_{R_{y}(\theta)}^{A_{0}\rightarrow B} = |0\rangle_{A}\langle 0|\otimes R_{y}(\theta) + |1\rangle_{A}\langle 1|\otimes \mathbb{I}_{B},
\end{equation}
em que $R_{y}(\theta)=\begin{bmatrix} \cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2) \end{bmatrix}$. Com isso, obteremos o estado
\begin{align}
|\Psi_{2}\rangle & = C_{R_{y}(\theta)}^{A_{0}\rightarrow B}|\Psi_{1}\rangle \\
& = c_{0}'|0\rangle_{A}\otimes\big(\cos(\theta/2)|0\rangle_{B}+\sin(\theta/2)|1\rangle_{B} \big)+c_{1}'|1\rangle_{A}\otimes|0\rangle_{B}.
\end{align}

Depois, aplica-se outra $R_{y}$ controlada, mas com $|1\rangle$ como ativador:
\begin{equation}
C_{R_{y}(\phi)}^{A_{1}\rightarrow B} = |0\rangle_{A}\langle 0|\otimes \mathbb{I}_{B} + |1\rangle_{A}\langle 1|\otimes R_{y}(\phi).
\end{equation}
Assim, obtemos o estado
\begin{align}
|\Psi_{3}\rangle & = C_{R_{y}(\phi)}^{A_{1}\rightarrow B}|\Psi_{2}\rangle \\
& = c_{0}'|0\rangle_{A}\otimes\big(\cos(\theta/2)|0\rangle_{B}+\sin(\theta/2)|1\rangle_{B} \big)+c_{1}'|1\rangle_{A}\otimes\big(\cos(\phi/2)|0\rangle_{B}+\sin(\phi/2)|1\rangle_{B} \big) \\
& = \big(\cos(\theta/2)c_{0}'|0\rangle_{A} + \cos(\phi/2)c_{1}'|1\rangle_{A}\big)\otimes|0\rangle_{B} + \big(\sin(\theta/2)c_{0}'|0\rangle_{A} + \sin(\phi/2)c_{1}'|1\rangle_{A}\big)\otimes|1\rangle_{B} \\
& = (D_{1}U_{A}|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} + (D_{2}U_{A}|\psi_{0}\rangle_{A})\otimes|1\rangle_{B}.
\end{align}

Acima definimos os operadores 
\begin{align}
D_{1} & = \cos(\theta/2)|0\rangle_{A}\langle 0| + \cos(\phi/2)|1\rangle_{A}\langle 1|, \\
D_{2} & = \sin(\theta/2)|0\rangle_{A}\langle 0| + \sin(\phi/2)|1\rangle_{A}\langle 1|,
\end{align}
que são operadores gerais de um qubit, a menos de uma transformação unitária para generalizar os autovetores. Podemos garantir que esses operadores sejam positivos restringindo
\begin{equation}
\theta,\phi\in[0,\pi].
\end{equation}

Por isso, seguimos aplicando uma unitária qualquer $V_{1}$ controlada
\begin{equation}
C_{V_{1}}^{B_{0}\rightarrow A} = V_{1}\otimes|0\rangle_{B}\langle 0| + \mathbb{I}\otimes|1\rangle_{B}\langle 1|,
\end{equation}
obtendo o estado
\begin{align}
|\Psi_{4}\rangle & = C_{V_{1}}^{B_{0}\rightarrow A}|\Psi_{3}\rangle \\
& = (V_{1}D_{1}U_{A}|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} + (D_{2}U_{A}|\psi_{0}\rangle_{A})\otimes|1\rangle_{B}.
\end{align}

Por fim, aplicamos uma unitária qualquer $V_{2}$ controlada
\begin{equation}
C_{V_{2}}^{B_{1}\rightarrow A} = \mathbb{I}_{A}\otimes|0\rangle_{B}\langle 0| + V_{2}\otimes|1\rangle_{B}\langle 1|,
\end{equation}
obtendo o estado
\begin{align}
|\Psi_{5}\rangle & = C_{V_{2}}^{B_{1}\rightarrow A}|\Psi_{4}\rangle \\
& = (V_{1}D_{1}U_{A}|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} + (V_{2}D_{2}U_{A}|\psi_{0}\rangle_{A})\otimes|1\rangle_{B} \\
& = (M_{1}|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} + (M_{2}|\psi_{0}\rangle_{A})\otimes|1\rangle_{B}.
\end{align}

Pelo teorema da decomposição em valores singulares, temos que para qualquer matriz $A$, existem matrizes unitárias $U$ e $W$ e uma matriz $D$ positiva e diagonal na base padrão $\{|j\rangle\}$ tais que
\begin{equation}
A = UDW.
\end{equation}
Portanto, o circuito acima é tal que temos duas matrizes gerais
\begin{equation}
M_{1} = V_{1}D_{1}U_{A} \text{ e } M_{2} = V_{2}D_{2}U_{A}.
\end{equation}
A possível limitação de $U_{A}$ aparecer nas duas equações é importante para que a relação de completeza dos operadores de medida seja satisfeita:
\begin{align}
\sum_{j}M_{j}^{\dagger}M_{j} & = M_{1}^{\dagger}M_{1} + M_{2}^{\dagger}M_{2} \\
& = U_{A}^{\dagger}D_{1}^{\dagger}V_{1}^{\dagger}V_{1}D_{1}U_{A} + U_{A}^{\dagger}D_{2}^{\dagger}V_{2}^{\dagger}V_{2}D_{2}U_{A} \\
& = U_{A}^{\dagger}D_{1}^{2}U_{A} + U_{A}^{\dagger}D_{2}^{2}U_{A} = U_{A}^{\dagger}\big(D_{1}^{2}+D_{2}^{2}\big)U_{A} \\
& = U_{A}^{\dagger}\big(\cos^{2}(\theta/2)|0\rangle_{A}\langle 0| + \cos^{2}(\phi/2)|1\rangle_{A}\langle 1| + \sin^{2}(\theta/2)|0\rangle_{A}\langle 0| + \sin^{2}(\phi/2)|1\rangle_{A}\langle 1|\big)U_{A} \\
& = U_{A}^{\dagger}\big(|0\rangle_{A}\langle 0| + |1\rangle_{A}\langle 1|\big)U_{A} = U_{A}^{\dagger}U_{A} \\
& = \mathbb{I}_{A}.
\end{align}

## POVMs de 1 qubit com $n$ elementos

Pode-se compor o circuito acima para implementar um POVM com $n$ elementos, como mostrado na figura abaixo:

<img src = 'qcircuit_povmn.png' width = '800'>

Vamos fazer as contas para $n=3$. Depois da aplicação da porta $U$ e do módulo 1, teremos obtido o estado
\begin{equation}
|\Phi_{1}\rangle = \big((V_{1}^{(1)}D_{1}^{(1)}U|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} + (V_{2}^{(1)}D_{2}^{(1)}U|\psi_{0}\rangle_{A})\otimes|1\rangle_{B}\big)\otimes|0\rangle_{C},
\end{equation}
com 
\begin{align}
& D_{1}^{(1)} = \cos(\theta_{1}/2)|0\rangle_{A}\langle 0| + \cos(\phi_{1}/2)|1\rangle_{A}\langle 1| \\
& D_{2}^{(1)} = \sin(\theta_{1}/2)|0\rangle_{A}\langle 0| + \sin(\phi_{1}/2)|1\rangle_{A}\langle 1|.
\end{align}

Seguindo, aplicamos a operação controlada
\begin{align}
C^{A_{0}B_{1}\rightarrow C}_{R_{y}(\theta_{2})} = |0\rangle_{A}\langle 0|\otimes|1\rangle_{B}\langle 1|\otimes R_{y}(\theta_{2}) + (|0\rangle_{A}\langle 0|\otimes|0\rangle_{B}\langle 0|+|1\rangle_{A}\langle 1|\otimes|0\rangle_{B}\langle 0|+|1\rangle_{A}\langle 1|\otimes|1\rangle_{B}\langle 1|)\otimes\mathbb{I}_{C}.
\end{align}

Obteremos assim o estado (usa $|\psi_{0}\rangle = c_{0}|0\rangle+c_{1}|1\rangle$)
\begin{align}
|\Phi_{2}\rangle & = C^{A_{0}B_{1}\rightarrow C}_{R_{y}(\theta_{2})}|\Phi_{1}\rangle \\
& = \big((V_{1}^{(1)}D_{1}^{(1)}U|\psi_{0}\rangle_{A})\otimes|0\rangle_{B} + (V_{2}^{(1)}D_{2}^{(1)}Uc_{1}|1\rangle_{A})\otimes|1\rangle_{B}\big)\otimes|0\rangle_{C} + (V_{2}^{(1)}D_{2}^{(1)}Uc_{0}|0\rangle_{A})\otimes|1\rangle_{B}\big)\otimes R_{y}(\theta_{2})|0\rangle_{C},
\end{align}

Agora, aplicamos
\begin{align}
C^{A_{1}B_{1}\rightarrow C}_{R_{y}(\phi_{2})} = |1\rangle_{A}\langle 1|\otimes|1\rangle_{B}\langle 1|\otimes R_{y}(\phi_{2}) + (|0\rangle_{A}\langle 0|\otimes|0\rangle_{B}\langle 0|+|1\rangle_{A}\langle 1|\otimes|0\rangle_{B}\langle 0|+|0\rangle_{A}\langle 0|\otimes|1\rangle_{B}\langle 1|)\otimes\mathbb{I}_{C}.
\end{align}

Obteremos assim o estado (usa $|\psi_{0}\rangle = c_{0}|0\rangle+c_{1}|1\rangle$)
\begin{align}
|\Phi_{3}\rangle & = C^{A_{1}B_{1}\rightarrow C}_{R_{y}(\phi_{2})}|\Phi_{3}\rangle \\
& = (V_{1}^{(1)}D_{1}^{(1)}U|\psi_{0}\rangle_{A})\otimes|00\rangle_{BC} + (V_{2}^{(1)}D_{2}^{(1)}Uc_{1}|1\rangle_{A})\otimes|1\rangle_{B}\otimes R_{y}(\phi_{2})|0\rangle_{C} + (V_{2}^{(1)}D_{2}^{(1)}Uc_{0}|0\rangle_{A})\otimes|1\rangle_{B}\big)\otimes R_{y}(\theta_{2})|0\rangle_{C} \\
& = (V_{1}^{(1)}D_{1}^{(1)}U|\psi_{0}\rangle_{A})\otimes|00\rangle_{BC} + (V_{2}^{(1)}D_{2}^{(1)}Uc_{1}|1\rangle_{A})\otimes|1\rangle_{B}\otimes\big(\cos(\phi_{2}/2)|0\rangle_{C}+\sin(\phi_{2}/2)|1\rangle_{C}\big) \\
& + (V_{2}^{(1)}D_{2}^{(1)}Uc_{0}|0\rangle_{A})\otimes|1\rangle_{B}\big)\otimes\big(\cos(\theta_{2}/2)|0\rangle_{C}+\sin(\theta_{2}/2)|1\rangle_{C}\big) \\
& = V_{1}^{(1)}D_{1}^{(1)}U|\psi_{0}\rangle_{A}\otimes|00\rangle_{BC} + V_{2}^{(1)}D_{2}^{(1)}U\big(\cos(\theta_{2}/2)c_{0}|0\rangle_{A}+\cos(\phi_{2}/2)c_{1}|1\rangle_{A}\big)\otimes|10\rangle_{BC} \\
&  + V_{2}^{(1)}D_{2}^{(1)}U\big(\sin(\theta_{2}/2)c_{0}|0\rangle_{A}+\sin(\phi_{2}/2)c_{1}|1\rangle_{A}\big)\otimes|11\rangle_{BC} \\
& = V_{1}^{(1)}D_{1}^{(1)}U|\psi_{0}\rangle_{A}\otimes|00\rangle_{BC} + V_{2}^{(1)}D_{2}^{(1)}UD_{1}^{(2)}|\psi_{0}\rangle_{A}\otimes|10\rangle_{BC} + V_{2}^{(1)}D_{2}^{(1)}UD_{2}^{(2)}|\psi_{0}\rangle_{A}\otimes|11\rangle_{BC}.
\end{align}

Aplicando $C_{V_{1}^{(2)}}^{B_{1}C_{0}\rightarrow A}$ e $C_{V_{2}^{(2)}}^{B_{1}C_{1}\rightarrow A}$ obteremos